In [ ]:
import os
import re
from datetime import datetime

import pandas as pd

pd.options.display.max_columns = None

In [ ]:
DATA_FOLDER = 'data'

INPUT_FILE_NAME = 'rules s1 a 5 mayo 24 excel.xlsx'
OUTPUT_FILE_NAME = 'acumulado_Campo_S1.csv'
NEW_OUTPUT_FILE_NAME = 'res.csv'

INPUT_FILE_PATH = os.path.join(DATA_FOLDER, INPUT_FILE_NAME)
OUTPUT_FILE_PATH = os.path.join(DATA_FOLDER, OUTPUT_FILE_NAME)
NEW_OUTPUT_FILE_PATH = os.path.join(DATA_FOLDER, NEW_OUTPUT_FILE_NAME)

In [ ]:
# Skip the first 3 rows as they only contain description and
# don't fit the structure of the rest of the file
df = pd.read_excel(INPUT_FILE_PATH, skiprows=3, decimal=',')

# Set the depth as the index of the dataframe for easier manipulation
df = df.rename(columns={'Depth (m)': 'Depth'}).set_index('Depth')

# Remove the second depth column as it's a duplicate and all empty columns
df = df.drop(columns=['Depth (m).1'] + df.filter(regex='Unnamed').columns.tolist())

In [ ]:
# Split the dataframe into two separate dataframes
# Asserts are used to make sure that the split is correct
n_columns = df.shape[1]

assert n_columns % 2 == 0, f'Number of columns is not even: {n_columns}'

df_A = df.iloc[:, :n_columns // 2]
df_B = df.iloc[:, n_columns // 2:]

assert df_A.shape == df_B.shape, 'Dataframes do not have the same shape'
assert df.index.equals(df_A.index), 'Indexes do not match'

In [ ]:
# A decorator that converts the returned date to a string
def date_to_string(fun: callable) -> callable:
    def wrapper(date: str | datetime) -> str:
        date = fun(date)
        if pd.isnull(date):
            return ''

        return date.strftime('%d/%m/%Y')

    return wrapper


@date_to_string
def process_date(date: str | datetime) -> pd.Timestamp:
    if isinstance(date, datetime):
        # The dates are wrongly read because of Excel's date formatting
        # so we swap the month and the day positions
        date_str = date.strftime('%m/%d/%Y')
        return pd.to_datetime(date_str, format='%d/%m/%Y')

    if isinstance(date, str):
        if (match := re.search(r'(\d+)/\d+/(\d+).*', date)):
            # Determine whether the format is dd/mm/yyyy or yyyy/mm/dd
            if len(match.group(1)) == 4:
                return pd.to_datetime(date, format='%Y/%m/%d', exact=False)

            if len(match.group(2)) == 4:
                return pd.to_datetime(date, format='%d/%m/%Y', exact=False)

    return pd.to_datetime(errors='coerce')

In [ ]:
df_res = pd.DataFrame(index=df.index)

for depth in df.index:
    for nth_col in range(n_columns // 2):
        # Take the nth column of the first dataframe and the nth column of the second dataframe
        # and put them into the resulting dataframe at the same depth.
        df_res.loc[depth, f'A{nth_col}'] = df_A.loc[depth].iloc[nth_col]
        df_res.loc[depth, f'B{nth_col}'] = df_B.loc[depth].iloc[nth_col]

        # The Excel outputted the dates in two different formats so this is a hack to unify it
        df_res.loc[depth, f'Date{nth_col}'] = process_date(df_A.columns[nth_col])

In [ ]:
# Check that all columns are the same as in the test data

df_test = pd.read_csv(OUTPUT_FILE_PATH, sep=';', decimal=',')
df_test = df_test.set_index('Depth')

for col in df_res.filter(regex='A|B\d+').columns:
    assert df_res[col].equals(df_test[col].astype('float64')), f'Columns do not match: {col}'

In [ ]:
df_res.to_csv(NEW_OUTPUT_FILE_PATH, sep=';', decimal=',', index=True)